In [2]:
import sqlite3
import pandas as pd

# Conexión (usa ':memory:' si no quieres archivo)
conn = sqlite3.connect("database_main.db")
cursor = conn.cursor()
conn.close()

In [3]:
conn = sqlite3.connect("database_main.db")
cursor = conn.cursor()

# Crear tabla DimPais
cursor.execute("""
                CREATE TABLE IF NOT EXISTS DimPais (
                PaisCode VARCHAR(10) PRIMARY KEY,
                PaisNombre VARCHAR(100)
                )
                """)
conn.commit()
conn.close()

In [4]:
conn = sqlite3.connect("database_main.db")
cursor = conn.cursor()

# Crear tabla DimPais
cursor.execute("""
                INSERT INTO DimPais VALUES 
                ('EC','Ecuador'),
                ('CO','Colombia'),
                ('PE','Peru')
                """)
conn.commit()
conn.close()

In [5]:
conn = sqlite3.connect("database_main.db")
cursor = conn.cursor()

# Crear tabla DimPais
cursor.execute("""
               CREATE TABLE Ventas (
               FactID INT IDENTITY PRIMARY KEY,
               PaisCode VARCHAR(10) NOT NULL,
               Fecha DATE NOT NULL,
               NetUSD DECIMAL(18,2) NOT NULL,
               TaxUSD DECIMAL(18,2) NOT NULL,
               FOREIGN KEY (PaisCode) REFERENCES DimPais(PaisCode)
)
                """)
conn.commit()
conn.close()

In [6]:
conn = sqlite3.connect("database_main.db")
cursor = conn.cursor()

# Crear tabla DimPais
cursor.execute("""
              INSERT INTO Ventas (PaisCode, Fecha, NetUSD, TaxUSD) VALUES
                ('EC','2024-05-01',1000,120),
                ('CO','2024-05-02',650,123.5),
                ('PE','2024-05-03',800,144),
                ('EC','2024-05-10',900,108)
                """)
conn.commit()
conn.close()

In [8]:
conn = sqlite3.connect("database_main.db")
query1 = """
    SELECT 
    p.PaisNombre,
    SUM(v.NetUSD) AS TotalNetUSD,
    SUM(v.TaxUSD) AS TotalTaxUSD,
    SUM(v.NetUSD + v.TaxUSD) AS TotalUSD
    FROM Ventas v
    JOIN DimPais p ON v.PaisCode = p.PaisCode
    GROUP BY p.PaisNombre;
"""
df1 = pd.read_sql_query(query1, conn)
print("Totales por país:")
print(df1)

Totales por país:
  PaisNombre  TotalNetUSD  TotalTaxUSD  TotalUSD
0   Colombia          650        123.5     773.5
1    Ecuador         1900        228.0    2128.0
2       Peru          800        144.0     944.0


In [9]:
conn = sqlite3.connect("database_main.db")
query2 = """
    SELECT 
    p.PaisNombre,
    STRFTIME('%Y-%m', v.Fecha) AS Periodo,
    SUM(v.NetUSD) AS NetUSD_Mes,
    SUM(v.TaxUSD) AS TaxUSD_Mes
    FROM Ventas v
    JOIN DimPais p ON v.PaisCode = p.PaisCode
    GROUP BY p.PaisNombre, STRFTIME('%Y-%m', v.Fecha)
    ORDER BY p.PaisNombre, Periodo;
"""
df2 = pd.read_sql_query(query2, conn)
print("\nTotales mes a mes:")
print(df2)


Totales mes a mes:
  PaisNombre  Periodo  NetUSD_Mes  TaxUSD_Mes
0   Colombia  2024-05         650       123.5
1    Ecuador  2024-05        1900       228.0
2       Peru  2024-05         800       144.0


In [10]:
conn = sqlite3.connect("database_main.db")
query3 = """
    SELECT 
    p.PaisNombre,
    ROUND(SUM(v.TaxUSD) / NULLIF(SUM(v.NetUSD), 0), 4) AS Ratio_Tax_Net
    FROM Ventas v
    JOIN DimPais p ON v.PaisCode = p.PaisCode
    GROUP BY p.PaisNombre;
"""
df3 = pd.read_sql_query(query3, conn)
print("\nRatio Tax/Net por país:")
print(df3)


Ratio Tax/Net por país:
  PaisNombre  Ratio_Tax_Net
0   Colombia           0.19
1    Ecuador           0.00
2       Peru           0.00


In [11]:

conn = sqlite3.connect("database_main.db")
cursor = conn.cursor()
cursor.execute("CREATE INDEX IF NOT EXISTS idx_fecha_ventas ON Ventas(Fecha);")
print("\n✅ Índice creado en columna Fecha.")
conn.commit()
conn.close()


✅ Índice creado en columna Fecha.
